<a href="https://colab.research.google.com/github/MlMauriciolopes/Faculdade-Impacta/blob/main/aula3_webmining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aplicação - Calssificação de opiniões do Kindle

In [1]:
import pandas as pd
dados = pd.read_excel('/content/drive/MyDrive/aula_web_mining/reviews.xlsx')
dados.shape

(6926, 4)

In [5]:
dados.estrelas.value_counts()

5    5000
4    1087
3     385
1     285
2     169
Name: estrelas, dtype: int64

In [2]:
from nltk.tokenize.treebank import TreebankWordTokenizer
# Outros atributos do TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
stemmer = nltk.stem.RSLPStemmer()
tokenizer = nltk.tokenize.RegexpTokenizer('[^\W\d]{3,}')

def stemmer_tokenizer(doc):
  tokens = tokenizer.tokenize(doc)
  tokens = [stemmer.stem(t) for t in tokens]
  return tokens

stopwords = [stemmer.stem(s) for s in nltk.corpus.stopwords.words('portuguese')]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    binary=False,
    norm='l1',
    use_idf=True,
    stop_words = stopwords,
    tokenizer = stemmer_tokenizer
)

vetor = vectorizer.fit_transform(dados.texto)
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))
print(vetor)

In [4]:
vetor.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
from sklearn.model_selection import train_test_split
x = vetor
y = dados.categoria

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, stratify=y, random_state=88)
x_train.shape, x_test.shape

((4848, 4889), (2078, 4889))

In [12]:
print(y_train.value_counts(),'\n', y_test.value_counts())

1    4261
0     587
Name: categoria, dtype: int64 
 1    1826
0     252
Name: categoria, dtype: int64


In [19]:
from sklearn import tree

dtree = tree.DecisionTreeClassifier(random_state=88)
dtree.fit(x_train, y_train)

y_predict = dtree.predict(x_test)
y_predict

array([0, 1, 1, ..., 1, 1, 1])

In [21]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_predict)
acc

0.8715110683349374

In [22]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(y_test, y_predict)
cf

array([[ 121,  131],
       [ 136, 1690]])

In [25]:
from sklearn.metrics import precision_recall_fscore_support

prf = precision_recall_fscore_support(y_test, y_predict)
prf

(array([0.47081712, 0.9280615 ]),
 array([0.48015873, 0.92552026]),
 array([0.47544204, 0.92678914]),
 array([ 252, 1826]))

In [27]:
texto = 'Meu Kindle Veio com a caixa toda amassada e a tela rompida ao lado, infelizmente não pude trocar pq estava refém do período letivo, tendo em vista que sou docente de uma universidade. Infelizmente tive que ficar com o Kindle avariado e sem a proteção contra água'

vt = vectorizer.transform([texto])
vt.toarray()


array([[0., 0., 0., ..., 0., 0., 0.]])

In [28]:
dtree.predict(vt)

array([0])

In [31]:
from sklearn import linear_model

percep = linear_model.Perceptron(alpha=0.01, random_state=88)
percep.fit(x_train, y_train)

Perceptron(alpha=0.01, random_state=88)

In [33]:
y_predict = percep.predict(x_test)
y_predict

array([1, 1, 1, ..., 1, 1, 1])

In [36]:
acc = accuracy_score(y_test, y_predict)
acc

0.8960538979788258

In [37]:
precision_recall_fscore_support(y_test, y_predict)

(array([0.56617647, 0.94573643]),
 array([0.61111111, 0.93537788]),
 array([0.58778626, 0.94052863]),
 array([ 252, 1826]))

In [47]:
percep.predict(x_test)

array([1, 1, 1, ..., 1, 1, 1])

In [49]:
y_predict = percep.predict(x_test)

In [44]:
from sklearn import neural_network

percep = neural_network.MLPClassifier(hidden_layer_sizes=1,alpha=0.01, random_state=88)
percep.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(alpha=0.01, hidden_layer_sizes=1, random_state=88)

In [45]:
percep.predict_proba(x_test)

array([[0.13214246, 0.86785754],
       [0.02399104, 0.97600896],
       [0.10747623, 0.89252377],
       ...,
       [0.05504273, 0.94495727],
       [0.00871451, 0.99128549],
       [0.01973673, 0.98026327]])

In [50]:
precision_recall_fscore_support(y_test, y_predict)

(array([0.83783784, 0.90518962]),
 array([0.24603175, 0.99342826]),
 array([0.3803681 , 0.94725849]),
 array([ 252, 1826]))

In [51]:
from sklearn.model_selection import cross_val_predict

mlp = neural_network.MLPClassifier(hidden_layer_sizes=1,alpha=0.01, random_state=88)

y_predict = cross_val_predict(mlp, x, y, cv=10)
prf = precision_recall_fscore_support(y, y_predict)
prf



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

(array([0.76455026, 0.91600489]),
 array([0.34445769, 0.98537868]),
 array([0.47493837, 0.9494262 ]),
 array([ 839, 6087]))

In [18]:
# proba (off)
# y_predict = dtree.predict_proba(x_test)
# y_predict

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])